# ASSIGNMENT 6

# SUMMARIZATION

In [39]:
# Importing library

from nltk.corpus import stopwords #you can remove stop words for speed
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

In [40]:
import spacy
nlp = spacy.load("el_core_news_sm")
text = "Η ανθρώπινη φύση πάντα έχει αναζητήσει την απάντηση σε ερωτήσεις που αγγίζουν την ουσία της ύπαρξής της. Ο πολιτισμός αναδύεται από τον αρχαϊκό του βάθος και διαμορφώνει τη συλλογική μας ταυτότητα. Κάθε γενιά κληρονομεί την ιστορία της πατρίδας της και την ενσωματώνει στη δική της πραγματικότητα. Ο διάλογος μεταξύ των γενεών είναι ένας σημαντικός τρόπος να διατηρούμε ζωντανή τη μνήμη των προγόνων μας και να αναπτύσσουμε την κριτική μας σκέψη. Το πνεύμα της εξερεύν"
doc = nlp(text)
paragraph = list(doc.sents)
sentences = []
for sentence in paragraph:
    print(sentence.text)
    sentences.append(sentence.text.replace("[^a-zA-Z]", " ").split(" "))




Η ανθρώπινη φύση πάντα έχει αναζητήσει την απάντηση σε ερωτήσεις που αγγίζουν την ουσία της ύπαρξής της.
Ο πολιτισμός αναδύεται από τον αρχαϊκό του βάθος και διαμορφώνει τη συλλογική μας ταυτότητα.
Κάθε γενιά κληρονομεί την ιστορία της πατρίδας της και την ενσωματώνει στη δική της πραγματικότητα.
Ο διάλογος μεταξύ των γενεών είναι ένας σημαντικός τρόπος να διατηρούμε ζωντανή τη μνήμη των προγόνων μας και να αναπτύσσουμε την κριτική μας σκέψη.
Το πνεύμα της εξερεύν


In [41]:
# Our data: a list of sentences

print("Sentences are ", sentences)

Sentences are  [['Η', 'ανθρώπινη', 'φύση', 'πάντα', 'έχει', 'αναζητήσει', 'την', 'απάντηση', 'σε', 'ερωτήσεις', 'που', 'αγγίζουν', 'την', 'ουσία', 'της', 'ύπαρξής', 'της.'], ['Ο', 'πολιτισμός', 'αναδύεται', 'από', 'τον', 'αρχαϊκό', 'του', 'βάθος', 'και', 'διαμορφώνει', 'τη', 'συλλογική', 'μας', 'ταυτότητα.'], ['Κάθε', 'γενιά', 'κληρονομεί', 'την', 'ιστορία', 'της', 'πατρίδας', 'της', 'και', 'την', 'ενσωματώνει', 'στη', 'δική', 'της', 'πραγματικότητα.'], ['Ο', 'διάλογος', 'μεταξύ', 'των', 'γενεών', 'είναι', 'ένας', 'σημαντικός', 'τρόπος', 'να', 'διατηρούμε', 'ζωντανή', 'τη', 'μνήμη', 'των', 'προγόνων', 'μας', 'και', 'να', 'αναπτύσσουμε', 'την', 'κριτική', 'μας', 'σκέψη.'], ['Το', 'πνεύμα', 'της', 'εξερεύν']]


In [42]:
# Function to calculate similarity (count the number of common words)

def sentence_similarity(sent1, sent2 ):
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
    all_words = list(set(sent1 + sent2))
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
    # build the vector for the first sentence
    for w in sent1:
        vector1[all_words.index(w)] += 1
    # build the vector for the second sentence
    for w in sent2:
        vector2[all_words.index(w)] += 1
    return 1 - cosine_distance(vector1, vector2)

In [43]:
# Create the similarity matrix 

similarity_matrix = np.zeros((len(sentences), len(sentences)))
for idx1 in range(len(sentences)):
    for idx2 in range(len(sentences)):
        if idx1 == idx2: #ignore if both are same sentences
            continue
        similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1],
sentences[idx2])
print("Smilarity matrix \n", similarity_matrix)

Smilarity matrix 
 [[0.         0.         0.33485541 0.08377078 0.11470787]
 [0.         0.         0.05572782 0.24397502 0.        ]
 [0.33485541 0.05572782 0.         0.11420805 0.31277162]
 [0.08377078 0.24397502 0.11420805 0.         0.        ]
 [0.11470787 0.         0.31277162 0.         0.        ]]


In [44]:
# Get the pagerank scores
# Step 3 - Rank sentences in similarity martix

sentence_similarity_graph = nx.from_numpy_array(similarity_matrix)
scores = nx.pagerank(sentence_similarity_graph)
print("scores", scores)

scores {0: 0.20459929911077915, 1: 0.13566618846904896, 2: 0.3055154815934063, 3: 0.18746739525334463, 4: 0.16675163557342107}


In [45]:
# Sort sentences by pagerank
# Step 4 - Sort the rank and pick top sentences

ranked_sentence = sorted(((scores[i],s) for i,s in
enumerate(sentences)), reverse=True)
print("Indexes of top ranked_sentence order are \n\n",
ranked_sentence)

Indexes of top ranked_sentence order are 

 [(0.3055154815934063, ['Κάθε', 'γενιά', 'κληρονομεί', 'την', 'ιστορία', 'της', 'πατρίδας', 'της', 'και', 'την', 'ενσωματώνει', 'στη', 'δική', 'της', 'πραγματικότητα.']), (0.20459929911077915, ['Η', 'ανθρώπινη', 'φύση', 'πάντα', 'έχει', 'αναζητήσει', 'την', 'απάντηση', 'σε', 'ερωτήσεις', 'που', 'αγγίζουν', 'την', 'ουσία', 'της', 'ύπαρξής', 'της.']), (0.18746739525334463, ['Ο', 'διάλογος', 'μεταξύ', 'των', 'γενεών', 'είναι', 'ένας', 'σημαντικός', 'τρόπος', 'να', 'διατηρούμε', 'ζωντανή', 'τη', 'μνήμη', 'των', 'προγόνων', 'μας', 'και', 'να', 'αναπτύσσουμε', 'την', 'κριτική', 'μας', 'σκέψη.']), (0.16675163557342107, ['Το', 'πνεύμα', 'της', 'εξερεύν']), (0.13566618846904896, ['Ο', 'πολιτισμός', 'αναδύεται', 'από', 'τον', 'αρχαϊκό', 'του', 'βάθος', 'και', 'διαμορφώνει', 'τη', 'συλλογική', 'μας', 'ταυτότητα.'])]


In [46]:
# Pick the top “n” sentences
# Step 5 - How many sentences to pick

n = int(input("How many sentences do you want in the summary? "))
#n=2
summarize_text = []
for i in range(n):
    summarize_text.append(" ".join(ranked_sentence[i][1]))

How many sentences do you want in the summary? 2


In [47]:
# Finish off by printing summary
# Step 6 - Offcourse, output the summarize text

print("Summarize Text: \n", ". ".join(summarize_text))

Summarize Text: 
 Κάθε γενιά κληρονομεί την ιστορία της πατρίδας της και την ενσωματώνει στη δική της πραγματικότητα.. Η ανθρώπινη φύση πάντα έχει αναζητήσει την απάντηση σε ερωτήσεις που αγγίζουν την ουσία της ύπαρξής της.
